# Week 2: Dealing with data

Welcome back everyone!

The agenda for today:

- Review homework assignments, answer questions
- How is data stored?
- An lightning intro to 2 very important libraries: numpy and pandas

## Homework solutions



## How is data stored?

Lots of different ways of answering this question...

Moving from abstract to concrete:

- A brief intro to computer architecture
- Data storage paradigms
- Different types of files, the advantages and disadvantages of each

### How does the computer *physically* store data?

Quick intro to the modern CPU goes here.

The computer memory hierarchy:

![](images/ComputerMemoryHierarchy.png)

What does that mean for us:

- For large organizations, real tape backup is a thing! https://aws.amazon.com/glacier/
- Most of us store data long-term on hard drives
- When actively working on a project, our data lives in RAM
- When in the middle of a computation, data is shifting from RAM to CPU cache
- CPU actually does work on bits that are in its registers